In [7]:
"""
A random forest classifier aimed at determining whether a stock will be higher or lower after some given amount of days.
Replication of Khaidem, Saha, & Roy Dey (2016)

Documentation on function:
http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
"""

import pandas as pd
from sklearn.ensemble import RandomForestClassifier as make_forest
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import accuracy_score as acc
import numpy as np

'''
### Outline ###
We have a bunch of columns of different length target values
We drop all target values except the ones we want to analyze (or else when we remove NA we will remove too much data)
We then input the data and features in to the first .fit parameter, and the labels in the second
'''

num_features = 5
prediction_window = 1
full_data = pd.read_csv('data_preprocessed.csv')

In [8]:
# drop all target columns not to be analyzed
headers = full_data.columns.values
headers = headers[13:] # should return just the headers of the target values
headers = headers[headers!='Target({})'.format(prediction_window)]
selected_data = full_data.drop(headers, axis=1)
selected_data = selected_data.dropna(axis=0, how='any') # using the subset parameter might allow us to skip dropping other targets?

In [9]:
### Drop useless labels ###
selected_data.drop(["Unnamed: 0"], axis = 1, inplace = True)
selected_data.drop(["Date"], axis = 1, inplace = True)
selected_data.drop(["Open","High","Low"], axis = 1, inplace = True)
#selected_data.drop(["Symbol","Open","High","Low"], axis = 1, inplace = True)


In [25]:
def train_on_stock(stockname, df, train_frac,prediction_window=30):
    train_labels = ["Close","Volume","EWMA", "SO","WR","RSI"]
    #PROC
    symbole_df = df[df["Symbol"]==stockname]
    msk = np.random.rand(len(symbole_df)) < train_frac

    train_df = symbole_df[msk]
    test_df = symbole_df[~msk]
    
    train_y = train_df['Target({})'.format(prediction_window)].as_matrix()
    train_x = train_df[train_labels].as_matrix()
    
    test_y = test_df['Target({})'.format(prediction_window)].as_matrix()
    test_x =test_df[train_labels].as_matrix()
    
    
    Random_Forest = make_forest(n_estimators=65, max_features=6, bootstrap=True, oob_score=True, verbose=1,n_jobs=-1)
    Random_Forest.fit(train_x, train_y)
        
    res = Random_Forest.predict(test_x)
    test_accurency = Random_Forest.score(test_x, test_y)
    return Random_Forest, test_accurency

def train_on_market(df,train_frac,prediction_windows=30):
    train_labels = ["Close","Volume","EWMA", "SO","WR","RSI"]
    #PROC
    msk = np.random.rand(len(df)) < train_frac
    
    train_df = df[msk]
    test_df = df[~msk]
    
    train_y = train_df['Target({})'.format(prediction_window)].as_matrix()
    train_x = train_df[train_labels].as_matrix()
    
    test_y = test_df['Target({})'.format(prediction_window)].as_matrix()
    test_x =test_df[train_labels].as_matrix()
    
    Random_Forest = make_forest(n_estimators=65, max_features=6, bootstrap=True, oob_score=True, verbose=1,n_jobs=-1)
    Random_Forest.fit(train_x, train_y)
        
    
    test_accurency = Random_Forest.score(test_x, test_y)
    return Random_Forest, test_accurency

def test_for_stock(stockname, df, forest, prediction_windows):
    train_labels = ["Close","Volume","EWMA", "SO","WR","RSI"]

    test_y = df['Target({})'.format(prediction_window)].as_matrix()
    test_x = df[train_labels].as_matrix()
    
    return forest.score(test_x,test_y)

In [20]:
for stock in selected_data['Symbol'].unique():
    forest, accurency = train_on_stock(stock,selected_data,0.8,prediction_window)
    

[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  65 out of  65 | elapsed:    0.2s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  65 out of  65 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  65 out of  65 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  65 out of  65 | elapsed:    0.1s finished


(<361x24321 sparse matrix of type '<class 'numpy.int64'>'
	with 225399 stored elements in Compressed Sparse Row format>, array([    0,   407,   776,  1179,  1576,  1965,  2348,  2725,  3084,
        3437,  3820,  4211,  4592,  4975,  5360,  5713,  6100,  6491,
        6858,  7219,  7590,  7949,  8348,  8715,  9074,  9443,  9808,
       10175, 10534, 10893, 11264, 11633, 12020, 12397, 12778, 13179,
       13574, 13953, 14286, 14643, 15028, 15397, 15728, 16091, 16466,
       16841, 17250, 17639, 17994, 18351, 18724, 19079, 19466, 19853,
       20224, 20611, 20986, 21339, 21708, 22077, 22454, 22841, 23198,
       23555, 23924, 24321]))


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  65 out of  65 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  65 out of  65 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  65 out of  65 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  65 out of  65 | elapsed:    0.1s finished


(<171x25907 sparse matrix of type '<class 'numpy.int64'>'
	with 103170 stored elements in Compressed Sparse Row format>, array([    0,   427,   822,  1249,  1652,  2077,  2478,  2867,  3258,
        3669,  4072,  4473,  4848,  5241,  5608,  6001,  6392,  6781,
        7208,  7597,  8000,  8403,  8794,  9193,  9576,  9971, 10372,
       10777, 11184, 11581, 11968, 12343, 12774, 13159, 13542, 13911,
       14310, 14703, 15114, 15513, 15932, 16335, 16730, 17143, 17542,
       17947, 18348, 18763, 19148, 19571, 19968, 20377, 20796, 21163,
       21574, 21947, 22346, 22731, 23108, 23529, 23922, 24317, 24684,
       25103, 25504, 25907]))


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  65 out of  65 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  65 out of  65 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  65 out of  65 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  65 out of  65 | elapsed:    0.1s finished


...

(<373x25109 sparse matrix of type '<class 'numpy.int64'>'
	with 216546 stored elements in Compressed Sparse Row format>, array([    0,   371,   746,  1129,  1528,  1903,  2286,  2619,  3002,
        3381,  3760,  4159,  4530,  4903,  5286,  5653,  6056,  6445,
        6818,  7215,  7584,  7977,  8396,  8731,  9122,  9503,  9884,
       10293, 10698, 11091, 11464, 11839, 12204, 12579, 12970, 13367,
       13776, 14189, 14568, 14969, 15374, 15797, 16200, 16571, 16968,
       17365, 17736, 18121, 18512, 18887, 19286, 19693, 20130, 20513,
       20906, 21295, 21690, 22059, 22394, 22767, 23184, 23551, 23942,
       24317, 24712, 25109]))


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  65 out of  65 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  65 out of  65 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  65 out of  65 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  65 out of  65 | elapsed:    0.1s finished


(<42x25889 sparse matrix of type '<class 'numpy.int64'>'
	with 24864 stored elements in Compressed Sparse Row format>, array([    0,   395,   796,  1213,  1628,  2027,  2412,  2817,  3200,
        3583,  3982,  4393,  4784,  5189,  5582,  5987,  6350,  6759,
        7134,  7533,  7936,  8319,  8716,  9109,  9524,  9929, 10310,
       10695, 11094, 11501, 11900, 12267, 12674, 13085, 13478, 13867,
       14286, 14673, 15062, 15445, 15868, 16277, 16660, 17047, 17460,
       17879, 18272, 18673, 19098, 19503, 19920, 20325, 20716, 21143,
       21566, 21933, 22324, 22695, 23108, 23515, 23918, 24293, 24668,
       25041, 25472, 25889]))


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  65 out of  65 | elapsed:    0.2s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s


KeyboardInterrupt: 

In [26]:
complete_forest, accurency = train_on_market(selected_data,0.8,prediction_window)


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done  65 out of  65 | elapsed:  2.6min finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    1.0s
[Parallel(n_jobs=4)]: Done  65 out of  65 | elapsed:    1.6s finished


NameError: name 'prediction_windows' is not defined

In [27]:
acc = test_for_stock("A",selected_data,complete_forest,prediction_window)
print(acc)

[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    5.3s
[Parallel(n_jobs=4)]: Done  65 out of  65 | elapsed:    8.0s finished


0.957121794596


In [29]:
forest, accurency = train_on_stock("A",selected_data,0.8,prediction_window)
print(accurency)

[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  65 out of  65 | elapsed:    0.2s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  65 out of  65 | elapsed:    0.0s finished


0.761538461538


[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  65 out of  65 | elapsed:    0.0s finished
